In [3]:
from tkinter import *
from tkinter import messagebox
import pyodbc 
conx = pyodbc.connect("Driver={SQL Server};Server=TUANDOMI;UID=sa;PWD=tuan@123;Database=Python;")
cursor=conx.cursor()
window = Tk()
window.geometry("720x530")
window.resizable(False, False)
window.title('Kiểm tra')

#def
def updata(cur):
    cursor.execute("select count(*) from vquestionlist")
    maxim=cursor.fetchall()
    maxi = maxim[0][0]
    if maxi == 0:
        label1['text'] = "0"
        question_text.delete(1.0,'end')
        answer1.delete(0.0,'end')
        answer2.delete(0.0,'end')
        answer3.delete(0.0,'end')
        answer4.delete(0.0,'end')
    else:
        cursor.execute("select top 1 * from vquestionlist where stt2 = ?",cur)
        row=cursor.fetchall()
        question_text.delete(1.0,'end')
        answer1.delete(0.0,'end')
        answer2.delete(0.0,'end')
        answer3.delete(0.0,'end')
        answer4.delete(0.0,'end')
        question_text.insert(1.0,row[0][2])
        answer1.insert(1.0,row[0][3])
        answer2.insert(1.0,row[0][4])
        answer3.insert(1.0,row[0][5])
        answer4.insert(1.0,row[0][6])
        label1['text'] = str(cur)+"/"+str(maxi)
        var1.set(row[0][9])
        Radiobutton1['state']='normal'
        Radiobutton2['state']='normal'
        Radiobutton3['state']='normal'
        Radiobutton4['state']='normal'
        answer1['bg']='white'
        answer2['bg']='white'
        answer3['bg']='white'
        answer4['bg']='white'
        if var1.get()=='1000' or var1.get()=='0100' or var1.get()=='0010' or var1.get()=='0001':
            Radiobutton1['state']='disabled'
            Radiobutton2['state']='disabled'
            Radiobutton3['state']='disabled'
            Radiobutton4['state']='disabled'
            if var1.get()=='1000':
                answer1['bg']=row[0][10]
            if var1.get()=='0100':
                answer2['bg']=row[0][10]
            if var1.get()=='0010':
                answer3['bg']=row[0][10]
            if var1.get()=='0001':
                answer4['bg']=row[0][10]
def refresh():
    cursor.execute("update questionlist set an2 =''")
    cursor.commit()
    results_text['text']=''
    cursor.execute("select count(*) from vquestionlist where tra_loi like N'%Đúng%'")
    socau = cursor.fetchall()
    res['text']='Số câu trả lời đúng: ' + str(socau[0][0])
    for data in cursor.execute("select tra_loi from vquestionlist"):
        results_text['text'] +=  data[0] +'\n'
    updata(1)
def aprevious1():
    cur = int(label1['text'].split('/')[0])
    if cur > 1:
        cur = cur -1
        updata(cur)
def anext1():
    cur = int(label1['text'].split('/')[0])
    maxi = int(label1['text'].split('/')[1])
    if cur == maxi:
        messagebox.showinfo("Thông báo",res['text']+'/'+ str(maxi))
    if cur < maxi:
        cur = cur +1
        updata(cur)

def chooses():
    cur = int(label1['text'].split('/')[0])
    chon = var1.get()
    cursor.execute("""update questionlist set an2 = ? from questionlist a 
                   join vquestionlist b on a.stt = b.stt where b.stt2 = ?""",chon, cur)
    cursor.commit()
    
    results_text['text']=''
    cursor.execute("select count(*) from vquestionlist where tra_loi like N'%Đúng%'")
    socau = cursor.fetchall()
    res['text']='Số câu trả lời đúng của bạn là: ' + str(socau[0][0])
    for data in cursor.execute("select tra_loi from vquestionlist"):
        results_text['text'] +=  data[0] +'\n'
    if var1.get()=='1000' or var1.get()=='0100' or var1.get()=='0010' or var1.get()=='0001':
        Radiobutton1['state']='disabled'
        Radiobutton2['state']='disabled'
        Radiobutton3['state']='disabled'
        Radiobutton4['state']='disabled'
        cursor.execute("select top 1 * from vquestionlist where stt2 = ?",cur)
        row=cursor.fetchall()
        if var1.get()=='1000':
            answer1['bg']=row[0][10]
        if var1.get()=='0100':
            answer2['bg']=row[0][10]
        if var1.get()=='0010':
            answer3['bg']=row[0][10]
        if var1.get()=='0001':
            answer4['bg']=row[0][10]
    anext1()
def ins():
    if pas.get() != 'admin' or var1.get() =='0000':
        messagebox.showwarning("Cảnh báo","Thiếu đáp án hoặc sai mật khẩu")
    else:
        cursor.execute("""insert questionlist values (?,?,?,?,?,?,'') """,str(question_text.get(0.1,'end'))
                   ,str(answer1.get(0.1,'end')),str(answer2.get(0.1,'end')),str(answer3.get(0.1,'end'))
                   ,str(answer4.get(0.1,'end')),var1.get())
        cursor.commit()  
        refresh()
def deletes():
    if pas.get() != 'admin':
        messagebox.showwarning("Cảnh báo","Sai mật khẩu")
    else:
        cur = int(label1['text'].split('/')[0])
        cursor.execute("""delete questionlist from questionlist a 
                   join vquestionlist b on a.stt = b.stt where b.stt2 = ?""", cur)
        cursor.commit()
        refresh()
    
#frame
question = LabelFrame(window, text='Câu hỏi')
answers=LabelFrame(window, text='Các đáp án')
action = LabelFrame(window, text='Thao tác')
results = LabelFrame(window, text='Kết quả')

question.place(x=10,y=10,height=200,width=400)
answers.place(x=10,y=210,height=210,width=400)
action.place(x=10,y=420,height=100,width=400)
results.place(x=410,y=10,height=510,width=300)

#question
question_text = Text(question,font=('arial','14'))
question_text.pack(expand=True,fill=BOTH)

#answers
var1 = StringVar()
var1.set('0000')
Radiobutton1 = Radiobutton(answers, variable = var1,value = '1000')
Radiobutton2 = Radiobutton(answers, variable = var1,value = '0100')
Radiobutton3 = Radiobutton(answers, variable = var1,value = '0010')
Radiobutton4 = Radiobutton(answers, variable = var1,value = '0001')
Radiobutton1.grid(row=0,column=0,padx=5,pady=5)
Radiobutton2.grid(row=1,column=0,padx=5,pady=5)
Radiobutton3.grid(row=2,column=0,padx=5,pady=5)
Radiobutton4.grid(row=3,column=0,padx=5,pady=5)

answer1=Text(answers,height=2,width=43,font=('arial','11'))
answer2=Text(answers,height=2,width=43,font=('arial','11'))
answer3=Text(answers,height=2,width=43,font=('arial','11'))
answer4=Text(answers,height=2,width=43,font=('arial','11'))
answer1.grid(row=0,column=1,padx=5,pady=5)
answer2.grid(row=1,column=1,padx=5,pady=5)
answer3.grid(row=2,column=1,padx=5,pady=5)
answer4.grid(row=3,column=1,padx=5,pady=5)

#action 1
afresh = Button(action,text = "Làm mới",width=10,command = refresh)
aprevious = Button(action,text = "Câu hỏi trước",width=10,command = aprevious1)
label1=Label(action,width=3)
anext = Button(action,text = "Câu hỏi sau",width=10,command = anext1)
choose = Button(action,text = "Chọn",width=10,fg = 'red',command = chooses)

choose.grid(row=0,column=0,padx=5,pady=5)
aprevious.grid(row=0,column=1,padx=5,pady=5)
label1.grid(row=0,column=2,padx=5,pady=5)
anext.grid(row=0,column=3,padx=5,pady=5)
afresh.grid(row=0,column=4,padx=5,pady=5)

#action 2
add = Button(action,text = "Thêm câu hỏi",width=10,command = ins)
dele = Button(action,text = "Xóa câu hỏi",width=10,command = deletes)
label2=Label(action,text='Mật\nkhẩu',width=3)
pas=Entry(action,width=27,show='*')

add.grid(row=1,column=0,padx=5,pady=5)
dele.grid(row=1,column=1,padx=5,pady=5)
label2.grid(row=1,column=2,padx=5,pady=5)
pas.grid(row=1,column=3,columnspan=4,padx=5,pady=5)

#results
res = Label(results,font = ('bold'),fg = 'red')
res.place(x=5,y=1)

results_text = Label(results,justify=LEFT)
results_text.place(x=5,y=20)
cursor.execute("select count(*) from vquestionlist where tra_loi like N'%Đúng%'")
socau = cursor.fetchall()
res['text']='Số câu trả lời đúng của bạn là: ' + str(socau[0][0])
for data in cursor.execute("select tra_loi from vquestionlist"):
    results_text['text'] +=  data[0] +'\n'

#Update
updata(1)

window.mainloop()